## yt-napari updates! 

(pyenv activate yt_dev)

In [ ]:
import napari

In [4]:
viewer = napari.Viewer()  # load yt_scratch/notebooks/dxl20220209/iso_galaxy.json

json contents:

```json
{"$schema": "yt-napari_0.0.1.json",
  "dataset": "IsolatedGalaxy/galaxy0030/galaxy0030",
  "field_type": "enzo",
  "field_name": "Density",
  "left_edge": [0.45, 0.45, 0.45],
  "right_edge": [0.55, 0.55, 0.55],
  "edge_units": "code_length",
  "resolution": [400, 400, 400],
  "take_log": true}
```


In [2]:
# open another
viewer.layers.clear()
viewer.open("./amrvac3d_eyeofsauron.json")
viewer.layers[0].colormap = "inferno"
viewer.dims.ndisplay = 3

yt : [INFO     ] 2022-02-08 14:42:31,331 Parameters: current_time              = 0.02
yt : [INFO     ] 2022-02-08 14:42:31,332 Parameters: domain_dimensions         = [16 16 16]
yt : [INFO     ] 2022-02-08 14:42:31,333 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2022-02-08 14:42:31,333 Parameters: domain_right_edge         = [2. 2. 2.]
yt : [INFO     ] 2022-02-08 14:42:31,334 Parameters: cosmological_simulation   = 0
yt : [INFO     ] 2022-02-08 14:42:31,355 Using full MHD energy for thermal pressure.


# how it works

### Napari's new plugin engine (`npe2`) 

yaml specification of plugin hooks and source code location. `napari.yaml` (in `yt-napari/src/yt_napari`):

```yaml
name: yt-napari
display_name: yt-napari
contributions:
  commands:
    - id: yt-napari.get_reader
      python_name: yt_napari._reader:napari_get_reader
      title: Open data with napari FooBar  
  readers:
    - command: yt-napari.get_reader
      accepts_directories: false
      filename_patterns: ['*.json']
```


### The schema approach

Currently a vast simplification of ideas in `analysis_schema` work:

1. Data model & Schema:
    * A pydantic class `yt_napari._data_model.InputModel`
    * generated schema in `yt_napari/schemas/yt-napari_x.x.x.json`
2. JSON ingestion: `yt_napari._reader.reader_function` redirects to `yt_napari._model_ingestor.load_from_json()`, 

```python
def load_from_json(json_path: str):

    # InputModel is a pydantic class, the following will validate the json
    model = InputModel.parse_file(json_path)

    # now that we have a validated model, we can use the model attributes to
    # execute the code that will actually return our array for the image
    return _process_validated_model(model)
```

`_process_validated_model` is where the yt commands get run. 

Will be easy to use `analysis_schema` directly when ready!




## coming soon...

For first beta release (https://github.com/data-exp-lab/yt-napari/issues/7):
* specify multiple fields to get multiple layers
* add 2d image layer functionality for slices
* image layer spatial referencing (napari works in pixel/image coords, yt in spatial, need to scale + translate)
* documentation

Concurrent with first release:
* On the yt side: add a `.to_napari` method to easily sample and add a field from a yt selection object to a napari viewer. 

Longer term:
* semi-automated generation of a napari plugin widget for loading data (convert the schema to field inputs)
* move to yt-project github
   